https://kean-chan.medium.com/real-time-facial-recognition-with-pytorch-facenet-ca3f6a510816

In [1]:
import os
import cv2
import torch
from facenet_pytorch import InceptionResnetV1, MTCNN
from tqdm import tqdm
from types import MethodType

### helper function
def encode(img):
    res = resnet(torch.Tensor(img))
    return res

def detect_box(self, img, save_path=None):
    # Detect faces
    batch_boxes, batch_probs, batch_points = self.detect(img, landmarks=True)
    # Select faces
    if not self.keep_all:
        batch_boxes, batch_probs, batch_points = self.select_boxes(
            batch_boxes, batch_probs, batch_points, img, method=self.selection_method
        )
    # Extract faces
    faces = self.extract(img, batch_boxes, save_path)
    return batch_boxes, faces


### load model
resnet = InceptionResnetV1(pretrained='vggface2').eval()
mtcnn = MTCNN(
  image_size=224, keep_all=True, thresholds=[0.4, 0.5, 0.5], min_face_size=60
)
mtcnn.detect_box = MethodType(detect_box, mtcnn)

/home/alexander/Documents/Programming/ObjectDetection/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from pathlib import Path

for p in Path('../data/images').rglob('*.jpg'):
    print(f"{p.name}:\n{p}")

photo_1.jpg:
../data/images/photo_1.jpg
ZowjqYGdvbg_1_1.jpg:
../data/images/ZowjqYGdvbg_1_1.jpg
IMG_20221130_171901_617_1.jpg:
../data/images/IMG_20221130_171901_617_1.jpg
IMG_20230201_000037_1.jpg:
../data/images/IMG_20230201_000037_1.jpg
IMG_20230131_235814_1.jpg:
../data/images/IMG_20230131_235814_1.jpg
7Dy2eue2_3I_1.jpg:
../data/images/7Dy2eue2_3I_1.jpg
MicrosoftTeams-image (2)_1.jpg:
../data/images/MicrosoftTeams-image (2)_1.jpg
0je5b4V4MQ0_1.jpg:
../data/images/0je5b4V4MQ0_1.jpg
mBpuNzIMJFY.jpg:
../data/images/mBpuNzIMJFY.jpg
IMG_20230201_000034_1.jpg:
../data/images/IMG_20230201_000034_1.jpg
IMG_20230131_235833_1.jpg:
../data/images/IMG_20230131_235833_1.jpg
photo3_medium_1.jpg:
../data/images/photo3_medium_1.jpg
w_7fa4706c_1_1.jpg:
../data/images/w_7fa4706c_1_1.jpg
AD87rlDaNs8_1.jpg:
../data/images/AD87rlDaNs8_1.jpg
photo2-1_1.jpg:
../data/images/photo2-1_1.jpg
Kyz_E2PFg_g.jpg:
../data/images/Kyz_E2PFg_g.jpg
h5Xg1s6rMoQ.jpg:
../data/images/h5Xg1s6rMoQ.jpg
IMG_20220606_172837_1.

In [7]:
### get encoded features for all saved images
# saved_pictures = "../data/images"
all_people_faces = {}
# for file in saved_pictures:
for file in Path('../data/images').rglob('*.jpg'):
    person_face, extension = file.name.split(".")
    img = cv2.imread(f'{file}')
    cropped = mtcnn(img)
    if cropped is not None:
        all_people_faces[person_face] = encode(cropped)[0, :]

In [8]:
def detect(cam=0, thres=0.7):
    vdo = cv2.VideoCapture(cam)
    while vdo.grab():
        _, img0 = vdo.retrieve()
        batch_boxes, cropped_images = mtcnn.detect_box(img0)

        if cropped_images is not None:
            for box, cropped in zip(batch_boxes, cropped_images):
                x, y, x2, y2 = [int(x) for x in box]
                img_embedding = encode(cropped.unsqueeze(0))
                detect_dict = {}
                for k, v in all_people_faces.items():
                    detect_dict[k] = (v - img_embedding).norm().item()
                min_key = min(detect_dict, key=detect_dict.get)

                if detect_dict[min_key] >= thres:
                    min_key = 'Undetected'
                
                cv2.rectangle(img0, (x, y), (x2, y2), (0, 0, 255), 2)
                cv2.putText(
                  img0, min_key, (x + 5, y + 10), 
                   cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 255, 255), 1)
                
        ### display
        cv2.imshow("output", img0)
        if cv2.waitKey(1) == ord('q'):
            cv2.destroyAllWindows()
            break

detect()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/alexander/Documents/Programming/ObjectDetection/.venv/lib/python3.10/site-packages/cv2/qt/plugins"
